In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro")
    recall = recall_score(y_true, y_pred, average="macro")
    f1 = f1_score(y_true, y_pred, average="macro")
    return accuracy, precision, recall, f1

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder('train', transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=8)
val_data = datasets.ImageFolder('valid', transform=transform)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True, num_workers=8)

writer = SummaryWriter(log_dir="runs/alexnet")

model = models.alexnet(pretrained=True)
for par in model.parameters():
    par.requires_grad=False

model.classifier[6] = nn.Linear(model.classifier[6].in_features, 38)

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")


model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

num_epochs = 20
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
    
    accuracy, precision, recall, f1 = calculate_metrics(all_labels, all_preds)
    
    print(f"Epoch {epoch}/{num_epochs}: Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 score: {f1:.4f}")
    writer.add_scalar("Training Loss", total_loss, epoch)
    writer.add_scalar("Accuracy", accuracy, epoch)
    writer.add_scalar("Precision", precision, epoch)
    writer.add_scalar("Recall", recall, epoch)
    writer.add_scalar("F1 Score", f1, epoch)
    
    scheduler.step()
    
    # Validation
    model.eval()
    val_labels = []
    val_preds = []
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())
    val_accuracy, val_precision, val_recall, val_f1 = calculate_metrics(val_labels, val_preds)
    print(f"Validation: Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")
    writer.add_scalar("Validation Loss", val_loss, epoch)
    writer.add_scalar("Validation Accuracy", val_accuracy, epoch)
    writer.add_scalar("Validation Precision", val_precision, epoch)
    writer.add_scalar("Validation Recall", val_recall, epoch)
    writer.add_scalar("Validation F1 Score", val_f1, epoch)

# Save the model
torch.save(model.state_dict(), 'Alexnet_model.pth')

writer.close()


c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to C:\Users\LENOVO/.cache\torch\hub\checkpoints\alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [01:53<00:00, 2.15MB/s] 


GPU is not available
Epoch 1/20: Loss: 638.0319, Accuracy: 0.8199, Precision: 0.8192, Recall: 0.8201, F1 score: 0.8194
Validation: Loss: 77.8796, Accuracy: 0.9055, Precision: 0.9107, Recall: 0.9062, F1: 0.9039
Epoch 2/20: Loss: 466.7012, Accuracy: 0.8645, Precision: 0.8640, Recall: 0.8647, F1 score: 0.8643
Validation: Loss: 62.6348, Accuracy: 0.9246, Precision: 0.9285, Recall: 0.9249, F1: 0.9233
Epoch 3/20: Loss: 443.4666, Accuracy: 0.8727, Precision: 0.8724, Recall: 0.8730, F1 score: 0.8727
Validation: Loss: 65.2028, Accuracy: 0.9195, Precision: 0.9272, Recall: 0.9204, F1: 0.9192
Epoch 4/20: Loss: 431.2574, Accuracy: 0.8785, Precision: 0.8782, Recall: 0.8786, F1 score: 0.8784
Validation: Loss: 62.9648, Accuracy: 0.9275, Precision: 0.9310, Recall: 0.9276, F1: 0.9265
Epoch 5/20: Loss: 429.2622, Accuracy: 0.8814, Precision: 0.8813, Recall: 0.8816, F1 score: 0.8814
Validation: Loss: 59.3454, Accuracy: 0.9317, Precision: 0.9353, Recall: 0.9318, F1: 0.9309
Epoch 6/20: Loss: 425.5922, Accura

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro")
    recall = recall_score(y_true, y_pred, average="macro")
    f1 = f1_score(y_true, y_pred, average="macro")
    return accuracy, precision, recall, f1

transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    transforms.Grayscale(num_output_channels=1)
])

# Load dataset
train_data = datasets.ImageFolder('train', transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=8)
val_data = datasets.ImageFolder('valid', transform=transform)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True, num_workers=8)

writer = SummaryWriter(log_dir="runs/LeNet")


if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")

# Move model to device
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1,out_channels= 6,kernel_size= 5)
        self.pool = nn.AvgPool2d(kernel_size=2,stride=2)
        self.conv2 = nn.Conv2d(in_channels=6,out_channels= 16,kernel_size= 5)
        self.conv3 = nn.Conv2d(in_channels=16,out_channels= 120,kernel_size= 5)
        self.fc1=nn.Linear(120,84)
        self.fc2=nn.Linear(84,38)
    def forward(self,X):
        X=torch.tanh(self.conv1(X))
        X=self.pool(X)
        X=torch.tanh(self.conv2(X))
        X=self.pool(X)
        X=torch.tanh(self.conv3(X))
        X = X.view(-1, 120)
        X=torch.tanh(self.fc1(X))
        X=self.fc2(X)
        return X
model=LeNet()
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

num_epochs = 20
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
    
    accuracy, precision, recall, f1 = calculate_metrics(all_labels, all_preds)
    
    print(f"Epoch {epoch}/{num_epochs}: Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 score: {f1:.4f}")
    writer.add_scalar("Training Loss", total_loss, epoch)
    writer.add_scalar("Accuracy", accuracy, epoch)
    writer.add_scalar("Precision", precision, epoch)
    writer.add_scalar("Recall", recall, epoch)
    writer.add_scalar("F1 Score", f1, epoch)
    
    scheduler.step()
    
    # Validation
    model.eval()
    val_labels = []
    val_preds = []
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())
    val_accuracy, val_precision, val_recall, val_f1 = calculate_metrics(val_labels, val_preds)
    print(f"Validation: Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")
    writer.add_scalar("Validation Loss", val_loss, epoch)
    writer.add_scalar("Validation Accuracy", val_accuracy, epoch)
    writer.add_scalar("Validation Precision", val_precision, epoch)
    writer.add_scalar("Validation Recall", val_recall, epoch)
    writer.add_scalar("Validation F1 Score", val_f1, epoch)

# Save the model
torch.save(model.state_dict(), 'LeNet_model.pth')

writer.close()


GPU is not available
Epoch 1/20: Loss: 2857.2501, Accuracy: 0.2748, Precision: 0.2481, Recall: 0.2743, F1 score: 0.2473
Validation: Loss: 610.8152, Accuracy: 0.3597, Precision: 0.3546, Recall: 0.3599, F1: 0.3399
Epoch 2/20: Loss: 2221.0530, Accuracy: 0.4123, Precision: 0.3942, Recall: 0.4127, F1 score: 0.3956
Validation: Loss: 518.8904, Accuracy: 0.4470, Precision: 0.4445, Recall: 0.4488, F1: 0.4375
Epoch 3/20: Loss: 1943.8122, Accuracy: 0.4750, Precision: 0.4607, Recall: 0.4759, F1 score: 0.4638
Validation: Loss: 469.8042, Accuracy: 0.4952, Precision: 0.4899, Recall: 0.4967, F1: 0.4859


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000025C15DBB560>
Traceback (most recent call last):
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py", line 1562, in _shutdown_workers
    if self._persistent_workers or self._workers_status[worker_id]:
                                   ^^^^^^^^^^^^^^^^^^^^
AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute '_workers_status'
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x0000025C15DBB560>
Traceback (most recent call last):
  File "c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "c:\Users\LENOVO\AppData\Local\Programs\P

Epoch 4/20: Loss: 1765.1591, Accuracy: 0.5214, Precision: 0.5095, Recall: 0.5226, F1 score: 0.5128
Validation: Loss: 432.7563, Accuracy: 0.5299, Precision: 0.5180, Recall: 0.5318, F1: 0.5196
Epoch 5/20: Loss: 1630.5730, Accuracy: 0.5532, Precision: 0.5435, Recall: 0.5544, F1 score: 0.5463
Validation: Loss: 403.1619, Accuracy: 0.5581, Precision: 0.5519, Recall: 0.5607, F1: 0.5522
Epoch 6/20: Loss: 1540.0691, Accuracy: 0.5753, Precision: 0.5668, Recall: 0.5765, F1 score: 0.5695
Validation: Loss: 389.3841, Accuracy: 0.5717, Precision: 0.5700, Recall: 0.5731, F1: 0.5622
Epoch 7/20: Loss: 1462.3181, Accuracy: 0.5938, Precision: 0.5861, Recall: 0.5950, F1 score: 0.5886
Validation: Loss: 375.2482, Accuracy: 0.5868, Precision: 0.5961, Recall: 0.5886, F1: 0.5847
Epoch 8/20: Loss: 1314.0995, Accuracy: 0.6354, Precision: 0.6296, Recall: 0.6367, F1 score: 0.6312
Validation: Loss: 347.1352, Accuracy: 0.6156, Precision: 0.6071, Recall: 0.6169, F1: 0.6096
Epoch 9/20: Loss: 1289.7757, Accuracy: 0.6425

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro")
    recall = recall_score(y_true, y_pred, average="macro")
    f1 = f1_score(y_true, y_pred, average="macro")
    return accuracy, precision, recall, f1

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
train_data = datasets.ImageFolder('train', transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=8)
val_data = datasets.ImageFolder('valid', transform=transform)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True, num_workers=8)

writer = SummaryWriter(log_dir="runs/efficientnet_b0")

model = models.efficientnet_b0(pretrained=True)
for par in model.parameters():
    par.requires_grad=False

model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(model.classifier[1].in_features, 38)  # Adjust number of output classes
)
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")

# Move model to device

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

num_epochs = 20
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
    
    accuracy, precision, recall, f1 = calculate_metrics(all_labels, all_preds)
    
    print(f"Epoch {epoch}/{num_epochs}: Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 score: {f1:.4f}")
    writer.add_scalar("Training Loss", total_loss, epoch)
    writer.add_scalar("Accuracy", accuracy, epoch)
    writer.add_scalar("Precision", precision, epoch)
    writer.add_scalar("Recall", recall, epoch)
    writer.add_scalar("F1 Score", f1, epoch)
    
    scheduler.step()
    
    # Validation
    model.eval()
    val_labels = []
    val_preds = []
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())
    val_accuracy, val_precision, val_recall, val_f1 = calculate_metrics(val_labels, val_preds)
    print(f"Validation: Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")
    writer.add_scalar("Validation Loss", val_loss, epoch)
    writer.add_scalar("Validation Accuracy", val_accuracy, epoch)
    writer.add_scalar("Validation Precision", val_precision, epoch)
    writer.add_scalar("Validation Recall", val_recall, epoch)
    writer.add_scalar("Validation F1 Score", val_f1, epoch)

# Save the model
torch.save(model.state_dict(), 'efficientnet_b0_model.pth')

writer.close()


GPU is not available


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate metrics
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro")
    recall = recall_score(y_true, y_pred, average="macro")
    f1 = f1_score(y_true, y_pred, average="macro")
    return accuracy, precision, recall, f1

# Data transformation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
train_data = datasets.ImageFolder('train', transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=8)
val_data = datasets.ImageFolder('valid', transform=transform)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True, num_workers=8)

# TensorBoard writer
writer = SummaryWriter(log_dir="runs/shufflenet_v2")

# Load pretrained ShuffleNetV2
model = models.shufflenet_v2_x1_0(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(model.fc.in_features, 38)  # Adjust number of output classes
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move model to device
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Training loop
num_epochs = 20
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

    accuracy, precision, recall, f1 = calculate_metrics(all_labels, all_preds)

    print(f"Epoch {epoch}/{num_epochs}: Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    writer.add_scalar("Training Loss", total_loss, epoch)
    writer.add_scalar("Accuracy", accuracy, epoch)
    writer.add_scalar("Precision", precision, epoch)
    writer.add_scalar("Recall", recall, epoch)
    writer.add_scalar("F1 Score", f1, epoch)

    scheduler.step()

    # Validation
    model.eval()
    val_labels = []
    val_preds = []
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())

    val_accuracy, val_precision, val_recall, val_f1 = calculate_metrics(val_labels, val_preds)
    print(f"Validation: Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")
    writer.add_scalar("Validation Loss", val_loss, epoch)
    writer.add_scalar("Validation Accuracy", val_accuracy, epoch)
    writer.add_scalar("Validation Precision", val_precision, epoch)
    writer.add_scalar("Validation Recall", val_recall, epoch)
    writer.add_scalar("Validation F1 Score", val_f1, epoch)

# Save the model
torch.save(model.state_dict(), 'shufflenet_v2_model.pth')

# Close TensorBoard writer
writer.close()


c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ShuffleNet_V2_X1_0_Weights.IMAGENET1K_V1`. You can also use `weights=ShuffleNet_V2_X1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/shufflenetv2_x1-5666bf0f80.pth" to C:\Users\LENOVO/.cache\torch\hub\checkpoints\shufflenetv2_x1-5666bf0f80.pth
100%|██████████| 8.79M/8.79M [00:02<00:00, 3.27MB/s]


Using device: cpu
Epoch 1/20: Loss: 2700.3600, Accuracy: 0.7550, Precision: 0.7916, Recall: 0.7518, F1: 0.7567
Validation: Loss: 435.4503, Accuracy: 0.8746, Precision: 0.8780, Recall: 0.8734, F1: 0.8697
Epoch 2/20: Loss: 1334.8693, Accuracy: 0.8822, Precision: 0.8813, Recall: 0.8814, F1: 0.8786
Validation: Loss: 243.9134, Accuracy: 0.8982, Precision: 0.8987, Recall: 0.8973, F1: 0.8950
Epoch 3/20: Loss: 855.2797, Accuracy: 0.9010, Precision: 0.8997, Recall: 0.9005, F1: 0.8989
Validation: Loss: 169.5827, Accuracy: 0.9163, Precision: 0.9165, Recall: 0.9157, F1: 0.9146
Epoch 4/20: Loss: 636.4080, Accuracy: 0.9136, Precision: 0.9125, Recall: 0.9132, F1: 0.9121
Validation: Loss: 129.0395, Accuracy: 0.9281, Precision: 0.9276, Recall: 0.9276, F1: 0.9267
Epoch 5/20: Loss: 507.8490, Accuracy: 0.9241, Precision: 0.9232, Recall: 0.9238, F1: 0.9231
Validation: Loss: 107.4477, Accuracy: 0.9342, Precision: 0.9335, Recall: 0.9336, F1: 0.9331
Epoch 6/20: Loss: 429.0293, Accuracy: 0.9304, Precision: 0.9

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro")
    recall = recall_score(y_true, y_pred, average="macro")
    f1 = f1_score(y_true, y_pred, average="macro")
    return accuracy, precision, recall, f1

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset
train_data = datasets.ImageFolder('train', transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=8)
val_data = datasets.ImageFolder('valid', transform=transform)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True, num_workers=8)

writer = SummaryWriter(log_dir="runs/Plant_disease_CNN")

class Plant_disease_CNN(nn.Module):
    def __init__(self, num_classes=38):
        super(Plant_disease_CNN, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Conv2d(64, 192, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            # Extra layers for deeper feature extraction
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 6 * 6, 4000),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(4000),
            nn.Dropout(0.5),
            nn.Linear(4000, 4000),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(4000),
            nn.Linear(4000, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Instantiate the model
model = Plant_disease_CNN(num_classes=38)



# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

num_epochs = 20
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
    
    accuracy, precision, recall, f1 = calculate_metrics(all_labels, all_preds)
    
    print(f"Epoch {epoch}/{num_epochs}: Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 score: {f1:.4f}")
    writer.add_scalar("Training Loss", total_loss, epoch)
    writer.add_scalar("Accuracy", accuracy, epoch)
    writer.add_scalar("Precision", precision, epoch)
    writer.add_scalar("Recall", recall, epoch)
    writer.add_scalar("F1 Score", f1, epoch)
    
    scheduler.step()
    
    # Validation
    model.eval()
    val_labels = []
    val_preds = []
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())
    val_accuracy, val_precision, val_recall, val_f1 = calculate_metrics(val_labels, val_preds)
    print(f"Validation: Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")
    writer.add_scalar("Validation Loss", val_loss, epoch)
    writer.add_scalar("Validation Accuracy", val_accuracy, epoch)
    writer.add_scalar("Validation Precision", val_precision, epoch)
    writer.add_scalar("Validation Recall", val_recall, epoch)
    writer.add_scalar("Validation F1 Score", val_f1, epoch)


writer.close()


GPU is not available
Epoch 1/20: Loss: 1719.5297, Accuracy: 0.5863, Precision: 0.5808, Recall: 0.5861, F1 score: 0.5826
Validation: Loss: 415.4273, Accuracy: 0.6561, Precision: 0.7272, Recall: 0.6598, F1: 0.6395
Epoch 2/20: Loss: 634.9696, Accuracy: 0.8237, Precision: 0.8227, Recall: 0.8239, F1 score: 0.8232
Validation: Loss: 2528.2674, Accuracy: 0.7864, Precision: 0.8148, Recall: 0.7851, F1: 0.7845
Epoch 3/20: Loss: 493.4263, Accuracy: 0.8604, Precision: 0.8599, Recall: 0.8607, F1 score: 0.8602
Validation: Loss: 1829.8780, Accuracy: 0.8605, Precision: 0.8785, Recall: 0.8610, F1: 0.8624
Epoch 4/20: Loss: 358.5760, Accuracy: 0.8974, Precision: 0.8972, Recall: 0.8976, F1 score: 0.8973
Validation: Loss: 20100.7028, Accuracy: 0.9001, Precision: 0.9045, Recall: 0.8998, F1: 0.9001
Epoch 5/20: Loss: 268.9303, Accuracy: 0.9215, Precision: 0.9214, Recall: 0.9216, F1 score: 0.9215
Validation: Loss: 50778.4195, Accuracy: 0.9183, Precision: 0.9254, Recall: 0.9174, F1: 0.9181
Epoch 6/20: Loss: 286.

In [1]:
# optimising the model

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.nn.utils import clip_grad_norm_
def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro")
    recall = recall_score(y_true, y_pred, average="macro")
    f1 = f1_score(y_true, y_pred, average="macro")
    return accuracy, precision, recall, f1

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder('train', transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=4)
val_data = datasets.ImageFolder('valid', transform=transform)
val_loader = DataLoader(val_data, batch_size=64, shuffle=True, num_workers=4)

writer = SummaryWriter(log_dir="runs/Disease_classifier")

class Disease_classifier(nn.Module):
    def __init__(self, num_classes=38):
        super(Disease_classifier, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Conv2d(64, 192, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            
            nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        
        self.classifier = nn.Sequential(
            nn.Linear(512 * 6 * 6, 4000),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(4000),
            nn.Dropout(0.5),
            nn.Linear(4000, 4000),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(4000),
            nn.Linear(4000, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Instantiate the model
model = Disease_classifier(num_classes=38)


if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")
model = model.to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=.50)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

num_epochs = 20
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    all_labels = []
    all_preds = []
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
        
        total_loss += loss.item()
        
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
    
    accuracy, precision, recall, f1 = calculate_metrics(all_labels, all_preds)
    
    print(f"Epoch {epoch}/{num_epochs}: Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 score: {f1:.4f}")
    writer.add_scalar("Training Loss", total_loss, epoch)
    writer.add_scalar("Accuracy", accuracy, epoch)
    writer.add_scalar("Precision", precision, epoch)
    writer.add_scalar("Recall", recall, epoch)
    writer.add_scalar("F1 Score", f1, epoch)
    
    scheduler.step()
    
    # Validation
    model.eval()
    val_labels = []
    val_preds = []
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())
    val_accuracy, val_precision, val_recall, val_f1 = calculate_metrics(val_labels, val_preds)
    print(f"Validation: Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")
    writer.add_scalar("Validation Loss", val_loss, epoch)
    writer.add_scalar("Validation Accuracy", val_accuracy, epoch)
    writer.add_scalar("Validation Precision", val_precision, epoch)
    writer.add_scalar("Validation Recall", val_recall, epoch)
    writer.add_scalar("Validation F1 Score", val_f1, epoch)
    
    break

# Save the model
torch.save(model.state_dict(), 'Disease_classifier_model.pth')

writer.close()


GPU is not available
Epoch 1/20: Loss: 3296.0747, Accuracy: 0.6716, Precision: 0.6675, Recall: 0.6716, F1 score: 0.6679
Validation: Loss: 796.8983, Accuracy: 0.8345, Precision: 0.8578, Recall: 0.8337, F1: 0.8354
Epoch 2/20: Loss: 2997.4088, Accuracy: 0.8825, Precision: 0.8820, Recall: 0.8825, F1 score: 0.8822
Validation: Loss: 747.2046, Accuracy: 0.8700, Precision: 0.8853, Recall: 0.8699, F1: 0.8698
Epoch 3/20: Loss: 2805.0564, Accuracy: 0.9543, Precision: 0.9541, Recall: 0.9542, F1 score: 0.9541
Validation: Loss: 700.4998, Accuracy: 0.9719, Precision: 0.9720, Recall: 0.9719, F1: 0.9717
Epoch 4/20: Loss: 2778.3398, Accuracy: 0.9695, Precision: 0.9694, Recall: 0.9695, F1 score: 0.9694
Validation: Loss: 691.8987, Accuracy: 0.9776, Precision: 0.9774, Recall: 0.9775, F1: 0.9774
Epoch 5/20: Loss: 2762.0398, Accuracy: 0.9781, Precision: 0.9780, Recall: 0.9782, F1 score: 0.9781
Validation: Loss: 688.7205, Accuracy: 0.9821, Precision: 0.9820, Recall: 0.9820, F1: 0.9820
Epoch 6/20: Loss: 2759.4

KeyboardInterrupt: 

In [2]:
torch.save(model.state_dict(), 'Disease_classifier_model.pth')


In [3]:
model

Disease_classifier(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): BatchNo

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.nn.utils import clip_grad_norm_

def calculate_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro")
    recall = recall_score(y_true, y_pred, average="macro")
    f1 = f1_score(y_true, y_pred, average="macro")
    return accuracy, precision, recall, f1

train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder('train', transform=train_transform)
val_data = datasets.ImageFolder('valid', transform=val_transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True, num_workers=8)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False, num_workers=8)

writer = SummaryWriter(log_dir="runs/Disease_classifier_optimized")

class DiseaseClassifier(nn.Module):
    def __init__(self, num_classes=38):
        super(DiseaseClassifier, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(64, 192, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(192),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),

            nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(384),
            nn.ReLU(inplace=True),

            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        
        self.classifier = nn.Sequential(
            nn.Linear(256 * 6 * 6, 2050),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(2050),
            nn.Dropout(0.5),
            nn.Linear(2050, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU is not available")

model = DiseaseClassifier(num_classes=38).to(device)

model = model.to(device)

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = optim.Adam(model.parameters(), lr=0.0005)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

num_epochs = 20
observation_epochs = 4   
best_val_loss = float('inf')
early_stop_counter = 0

for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    all_labels, all_preds = [], []
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        clip_grad_norm_(model.parameters(), 1.0) 
        optimizer.step()

        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
    
    accuracy, precision, recall, f1 = calculate_metrics(all_labels, all_preds)
    
    print(f"Epoch {epoch}/{num_epochs}: Loss: {total_loss:.4f}, Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 score: {f1:.4f}")
    writer.add_scalar("Training Loss", total_loss, epoch)
    writer.add_scalar("Accuracy", accuracy, epoch)
    writer.add_scalar("Precision", precision, epoch)
    writer.add_scalar("Recall", recall, epoch)
    writer.add_scalar("F1 Score", f1, epoch)
    

    
    model.eval()
    val_loss = 0
    val_labels, val_preds = [], []
    
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())
    
    val_accuracy, val_precision, val_recall, val_f1 = calculate_metrics(val_labels, val_preds)
    print(f"Validation: Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}, Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1: {val_f1:.4f}")
    writer.add_scalar("Validation Loss", val_loss, epoch)
    writer.add_scalar("Validation Accuracy", val_accuracy, epoch)
    writer.add_scalar("Validation Precision", val_precision, epoch)
    writer.add_scalar("Validation Recall", val_recall, epoch)
    writer.add_scalar("Validation F1 Score", val_f1, epoch)

    scheduler.step(val_loss)  

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), 'Disease_classifier.pth') 
    else:
        early_stop_counter += 1
    
    if early_stop_counter >= observation_epochs:
        print("Early stopping triggered. Training stopped.")
        break

writer.close()

GPU is not available


c:\Users\LENOVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/20: Loss: 2102.0331, Accuracy: 0.5783, Precision: 0.5720, Recall: 0.5786, F1 score: 0.5739
Validation: Loss: 465.0744, Accuracy: 0.7114, Precision: 0.7950, Recall: 0.7109, F1: 0.7000
Epoch 2/20: Loss: 1450.6397, Accuracy: 0.8024, Precision: 0.8011, Recall: 0.8026, F1 score: 0.8016
Validation: Loss: 373.6571, Accuracy: 0.8562, Precision: 0.8765, Recall: 0.8568, F1: 0.8569
Epoch 3/20: Loss: 1254.1983, Accuracy: 0.8716, Precision: 0.8710, Recall: 0.8717, F1 score: 0.8712
Validation: Loss: 299.8428, Accuracy: 0.9162, Precision: 0.9210, Recall: 0.9162, F1: 0.9147
Epoch 4/20: Loss: 1143.9595, Accuracy: 0.9071, Precision: 0.9070, Recall: 0.9071, F1 score: 0.9070
Validation: Loss: 370.1141, Accuracy: 0.8472, Precision: 0.8904, Recall: 0.8501, F1: 0.8466
Epoch 5/20: Loss: 1080.5482, Accuracy: 0.9257, Precision: 0.9256, Recall: 0.9256, F1 score: 0.9256
Validation: Loss: 276.9079, Accuracy: 0.9411, Precision: 0.9453, Recall: 0.9409, F1: 0.9405
Epoch 6/20: Loss: 1037.8634, Accuracy: 0.9389

In [4]:
model

DiseaseClassifier(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): BatchNor

In [5]:
torch.save(model.state_dict(), 'Disease_classifier.pth') 